In [1]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

MPLBACKEND environment variable cleared.
Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20660, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 20660 (delta 5), reused 7 (delta 3), pack-reused 20647 (from 2)
Receiving objects: 100% (20660/20660), 278.00 MiB | 16.41 MiB/s, done.
Resolving deltas: 100% (1001/1001), done.
Updating files: 100% (138/138), done.


In [2]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")
    finally:
      !nvidia-smi

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
is_kaggle = ("kaggle" in ENV_NAME)
is_colab = not is_kaggle
print_system_info()

os.environ["WANDB_API_KEY"] = wandb_key = load_secret("WANDB_API_KEY")
os.environ["HF_TOKEN"] = HF_TOKEN = load_secret('HF_TOKEN')
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = 1

# Now, these libraries will log in automatically
import wandb
import huggingface_hub

wandb.login()
huggingface_hub.login(token=os.environ["HF_TOKEN"])

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/

🔧 System Information
Python version: 3.12.12
PyTorch version: 2.9.0+cu126
CUDA version: 12.6
GPU count: 1
  GPU 0: Tesla T4
Fri Jan  2 07:20:21 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             1

TypeError: str expected, not int

In [3]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch==2.9 torchvision
# 4. Install other dependencies

print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown tqdm
!uv pip install wandb hf_transfer
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 77ms
Prepared 1 package in 85ms
Uninstalled 1 package in 70ms
Installed 1 package in 16ms
 - pip==24.1.2
 + pip==25.3
/content
Using Python 3.12.12 environment at: /usr
Audited 2 packages in 100ms

⬇️ Installing Dependencies (Manually fixed)...
  × No solution found when resolving dependencies:
  ╰─▶ Because torch==1.13.1 has no wheels with a matching Python ABI tag
      (e.g., `cp312`) and xformers==0.0.16 depends on torch==1.13.1, we can
      conclude that xformers==0.0.16 cannot be used.
      And because you require xformers==0.0.16, we can conclude that your
      requirements are unsatisfiable.

      hint: You require CPython 3.12 (`cp312`), but we only found wheels for
      `torch` (v1.13.1) with the following Python ABI tags: `cp37m`, `cp38`,
      `cp39`, `cp310`, `cp311`
Using Python 3.12.12 environment at: /usr
Resolved 47 packages in 339ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend ret

In [6]:
# KAGGLE CELL #1: Download checkpoint
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
import os
import sys
from tqdm.auto import tqdm
from pathlib import Path
os.chdir(OUTPUT_PATH)
import os
from pathlib import Path
def download_from_hf(
    repo_id: str,
    local_dir: str = "ckpt",
    allow_patterns=None,
    force_download: bool = False,
    repo_type: str = "model"
):
    if allow_patterns is None:
        allow_patterns = ["*.safetensors", "scr*"]
    print(f"📥 Downloading checkpoint from Hugging Face Hub to '{local_dir}'...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id=repo_id,
        local_dir=local_dir,
        repo_type=repo_type,
        allow_patterns=allow_patterns,
        force_download=force_download
    )
    print("\n✅ Download complete!")
    print(f"\n📂 Files in {local_dir}/:")
    for file in os.listdir(local_dir):
        if file.endswith(".safetensors"):
            size = os.path.getsize(os.path.join(local_dir, file)) / (1024**2)
            print(f"  ✓ {file} ({size:.2f} MB)")

Using Python 3.12.12 environment at: /usr
Resolved 13 packages in 28ms
Audited 13 packages in 0.41ms


In [ ]:
download_from_hf(
    repo_id="dzungpham/font-diffusion-weights",
    local_dir="ckpt",
    allow_patterns=["*.safetensors", "scr*"]
)

In [7]:
download_from_hf(
    repo_id="dzungpham/font-diffusion-generated-data",
    local_dir="NomTuTao",
    repo_type="dataset",
    allow_patterns=["*Nom*"]
)

📥 Downloading checkpoint from Hugging Face Hub to 'NomTuTao'...



Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Ds_10k_ChuNom_TuTao.txt: 0.00B [00:00, ?B/s]

Ds_300_ChuNom_TuTao.csv: 0.00B [00:00, ?B/s]

Top184_PureNomChar.xlsx:   0%|          | 0.00/15.8k [00:00<?, ?B/s]


✅ Download complete!

📂 Files in NomTuTao/:


In [8]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /content/.


In [9]:
print("Model files:")
!ls -larth {OUTPUT_PATH}/ckpt

Model files:
total 1.1G
drwxr-xr-x 3 root root 4.0K Jan  2 07:20 .cache
-rw-r--r-- 1 root root 4.6M Jan  2 07:20 content_encoder.safetensors
-rw-r--r-- 1 root root  79M Jan  2 07:20 style_encoder.safetensors
-rw-r--r-- 1 root root 384M Jan  2 07:21 total_model.safetensors
-rw-r--r-- 1 root root 301M Jan  2 07:21 unet.safetensors
-rw-r--r-- 1 root root 272M Jan  2 07:21 scr_210000.pth
drwxr-xr-x 3 root root 4.0K Jan  2 07:21 .
drwxr-xr-x 1 root root 4.0K Jan  2 07:23 ..


In [10]:
%cd {OUTPUT_PATH}
# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================
HF_USERNAME = "dzungpham"
# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN

/content

EXPORTING DATASET TO DISK

📥 Loading dataset from Hub...
   Repository: dzungpham/font-diffusion-generated-data
   Split: train_original
README.md: 3.05kB [00:00, 8.19MB/s]
data/train_original-00000-of-00001.parqu(…): 100% 117M/117M [00:04<00:00, 25.7MB/s]
data/train-00000-of-00001.parquet: 100% 77.1M/77.1M [00:04<00:00, 17.6MB/s]
data/val-00000-of-00001.parquet: 100% 3.65M/3.65M [00:02<00:00, 1.63MB/s]
Generating train_original split: 100% 8985/8985 [00:00<00:00, 18014.66 examples/s]
Generating train split: 100% 5760/5760 [00:00<00:00, 14617.74 examples/s]
Generating val split: 100% 357/357 [00:00<00:00, 20638.53 examples/s]
✓ Loaded dataset with 8985 samples from Hub

Exporting images from dataset...

🎨 Exporting images...
Exporting: 100%|███████████████████████████| 8985/8985 [00:26<00:00, 333.70it/s]
✓ Exported 599 content images
✓ Exported 8985 target images

💾 Saving results_checkpoint.json...
  ✓ Saved results_checkpoint.json (8985 generations)

📊 Metadata Statistics:


In [ ]:
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN

In [ ]:
# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val" \
  --token HF_TOKEN

In [11]:
print("Fonts currently in fonts/ folder")
!ls -lt FontDiffusion/fonts
print("Styles in style_images/ folder")
!ls -l FontDiffusion/styles_images

Fonts currently in fonts/ folder
total 332584
-rw-r--r-- 1 root root 26929728 Jan  2 07:20  NomNaTongLight2.ttf
-rw-r--r-- 1 root root 14574480 Jan  2 07:20  NomNaTongLight.ttf
-rw-r--r-- 1 root root 31729820 Jan  2 07:20  NomNaTong-Regular2.otf
-rw-r--r-- 1 root root 14574552 Jan  2 07:20  NomNaTong-Regular.ttf
-rw-r--r-- 1 root root  9424552 Jan  2 07:20  NomNaTong-Regular.otf
-rw-r--r-- 1 root root 12967288 Jan  2 07:20  HanaMinC.otf
-rw-r--r-- 1 root root 30739236 Jan  2 07:20  HanaMinB.ttf
-rw-r--r-- 1 root root 32201032 Jan  2 07:20  HanaMinB.otf
-rw-r--r-- 1 root root 22761228 Jan  2 07:20  HanaMinA.ttf
-rw-r--r-- 1 root root 31621108 Jan  2 07:20  HanaMinA.otf
-rw-r--r-- 1 root root 18202176 Jan  2 07:20 'Han-nom Minh 1.42.otf'
-rw-r--r-- 1 root root 19505228 Jan  2 07:20  Han-Nom-Khai-Regular-300623.ttf
-rw-r--r-- 1 root root 20368044 Jan  2 07:20 'Han-Nom Kai 1.00.otf'
-rw-r--r-- 1 root root 33815824 Jan  2 07:20 'HAN NOM B.ttf'
-rw-r--r-- 1 root root 21320444 Jan  2 07:20 'H

In [13]:
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
%cd {OUTPUT_PATH}
!accelerate launch --num_processes 1 \
    FontDiffusion/sample_batch.py \
    --characters "NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts/NomNaTong-Regular.otf" \
    --output_dir "my_dataset/train_original" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 600 \
    --end_line 800 \
    --batch_size 35 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

Using Python 3.12.12 environment at: /usr
Resolved 13 packages in 44ms
Audited 13 packages in 2ms
/content
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/content/FontDiffusion/src/dpm_solver/dpm_solver_pytorch.py:53: SyntaxWarning: invalid escape sequence '\h'
  The `alphas_cumprod` is the \hat{alpha_n} arrays in the notations of DDPM. Specifically, DDPMs assume that
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
📖 Reading character file: 100%|█████████████████████████████████| 201/201 [00:00<00:00, 771kfile/s]
✓ Verifying style images: 100%|███████████████████████████████| 15.0/15.0 [00:00<00:00, 80.1kfile/s]
error: XDG_RUNTIME_DIR 

In [ ]:
!find my_dataset/train_original/ContentImage -type f | wc -l
!find my_dataset/train_original/TargetImage -type f | wc -l

In [12]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --seed 42

2026-01-02 07:25:02,544 | INFO | ✓ Using source directory: my_dataset/train_original
2026-01-02 07:25:02,548 | INFO | 
2026-01-02 07:25:02,548 | INFO | FONTDIFFUSION VALIDATION SPLIT CREATOR
2026-01-02 07:25:02,549 | INFO | ============================================================
2026-01-02 07:25:02,549 | INFO | 
2026-01-02 07:25:02,549 | INFO | ANALYZING TRAINING DATA
2026-01-02 07:25:02,549 | INFO | ============================================================
2026-01-02 07:25:02,549 | INFO | 
🔍 Scanning content images...
Content images: 100% 599/599 [00:00<00:00, 1227951.17img/s]
2026-01-02 07:25:02,552 | INFO |   ✓ Found 599 content images
2026-01-02 07:25:02,553 | INFO | 
🔍 Scanning target images...
Styles: 100% 15/15 [00:00<00:00, 359.87style/s]
2026-01-02 07:25:02,595 | INFO |   ✓ Found 8985 valid target images
2026-01-02 07:25:02,595 | INFO | 
🔍 Validating content ↔ target pairs...
Validating pairs: 100%|██████████████████████████████████| 8985/8985 [00:00<00:00, 2051822.37p

In [ ]:
!uv pip install --upgrade pyarrow datasets

In [ ]:
# remove_unparseable_files.py

with open("my_dataset/unparseable_files.txt", "r", encoding="utf-8") as f:
    paths = [line.strip() for line in f if line.strip()]

import os

for path in paths:
    try:
        if os.path.exists(path):
            os.remove(path)
            print(f"Deleted: {path}")
        else:
            print(f"Not found: {path}")
    except Exception as e:
        print(f"Error deleting {path}: {e}")

In [ ]:
HF_USERNAME = "dzungpham"
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "train_original" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "train" \
  --token {HF_TOKEN}

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val" \
  --repo_id dzungpham/font-diffusion-generated-data \
  --split "val" \
  --token {HF_TOKEN}



FONTDIFFUSION DATASET CREATOR

Data dir: my_dataset/train_original
Repo: dzungpham/font-diffusion-generated-data
Push to Hub: True
✓ Validated directory structure
  Content images: my_dataset/train_original/ContentImage
  Target images: my_dataset/train_original/TargetImage
  Results checkpoint: my_dataset/train_original/results_checkpoint.json

BUILDING DATASET

✓ Loaded results_checkpoint.json
  Generations: 8985
  Characters: 599
  Styles: 15

🖼️  Loading 8985 image pairs...
Loading image pairs: 100%|██████████████| 8.98k/8.98k [00:07<00:00, 1.18kpair/s]
✓ Loaded 8985 samples

PUSHING TO HUB
Repository: dzungpham/font-diffusion-generated-data
Split: train_original
Map: 100%|████████████████████████| 8985/8985 [00:00<00:00, 15307.79 examples/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.25s/ shards]

✓ Successfully pushed to Hub!
  URL: https://huggingface.co/datasets/dzungpham/font-diffusion-generated-data

✅ COMPLETE!

FONTDIFFUSION DATASET CREATOR

Data 

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# TRAINING PHASE 1
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"
import wandb

MAX_TRAIN_STEPS = 1000 # @param {type:"integer"}
!accelerate launch FontDiffusion/train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_1" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
      \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
      \
    --train_batch_size=8 \
    --gradient_accumulation_steps=2 \
    --perceptual_coefficient=0.05 \
    --offset_coefficient=0.7 \
    --max_train_steps=${MAX_TRAIN_STEPS} \
    --ckpt_interval=${MAX_TRAIN_STEPS // 2} \
    --log_interval=50 \
      \
    --learning_rate=1e-4 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision="no"

Using Python 3.11.13 environment at: /usr
Resolved 12 packages in 78ms                                         
Audited 12 packages in 0.12ms
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydan

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# TRAINING PHASE 2
!wandb login

MAX_TRAIN_STEPS = 1000 # @param {type:"integer"}
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    \
    --train_batch_size=8 \
    --gradient_accumulation_steps=2 \
    --perceptual_coefficient=0.05 \
    --offset_coefficient=0.7 \
    --max_train_steps=${MAX_TRAIN_STEPS} \
    --ckpt_interval=${MAX_TRAIN_STEPS // 2} \
    --log_interval=50 \
    \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!python FontDiffusion/upload_models.py \
    --weights_dir "outputs/FontDiffuser" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")